# Question 1

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# Import data
df = pd.read_csv('/Users/zachary/Desktop/WINTER22/Geog490/geospatial-data-science/labs/lab5/seattle_house_prices.csv')


In [3]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat']))
gdf = gdf.set_crs(4326, allow_override=True)

# Reproject everything to UTM 10N (EPSG:32610)
gdf_utm = gdf.to_crs('EPSG:32610')

# How many houses are in this dataset?

In [7]:
gdf.count()

price          19451
bedrooms       19451
bathrooms      19451
sqft_living    19451
sqft_lot       19451
yr_built       19451
lat            19451
long           19451
geometry       19451
dtype: int64

### There are 19,451 houses in this data set

# How many features are there for predicting house price?

In [8]:
gdf.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,yr_built,lat,long,geometry
0,538000,3,2.25,2570,7242,1951,47.7210,-122.319,POINT (-122.31900 47.72100)
1,180000,2,1.00,770,10000,1933,47.7379,-122.233,POINT (-122.23300 47.73790)
2,604000,4,3.00,1960,5000,1965,47.5208,-122.393,POINT (-122.39300 47.52080)
3,510000,3,2.00,1680,8080,1987,47.6168,-122.045,POINT (-122.04500 47.61680)
4,1230000,4,4.50,5420,101930,2001,47.6561,-122.005,POINT (-122.00500 47.65610)


### As we can see, there are 5 features that could be used to predict housing prices. I guess lat and long could be considered 6 & 7 ( I think those are terrible variables), as the correlation question assumes those are included. My final answer is 7.

# Are there any null values in this dataset?

In [9]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19451 entries, 0 to 19450
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   price        19451 non-null  int64   
 1   bedrooms     19451 non-null  int64   
 2   bathrooms    19451 non-null  float64 
 3   sqft_living  19451 non-null  int64   
 4   sqft_lot     19451 non-null  int64   
 5   yr_built     19451 non-null  int64   
 6   lat          19451 non-null  float64 
 7   long         19451 non-null  float64 
 8   geometry     19451 non-null  geometry
dtypes: float64(3), geometry(1), int64(5)
memory usage: 1.3 MB


### There are no null values in the data set. We can tell this because the count of features is 19451, which matches the non_null count for each variable.

# Which three variables are best correlated with house price (include correlation coefficients)?
# Which three variables are least correlated with house price (include correlation coefficients)?

In [10]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix["price"].sort_values(ascending= False)

price          1.000000
sqft_living    0.702296
bathrooms      0.524395
bedrooms       0.315804
lat            0.308082
sqft_lot       0.090125
yr_built       0.052453
long           0.020092
Name: price, dtype: float64

### sqft_living is the most correlated (0.702296) followed by bathrooms (0.524395), and bedrooms (0.315804)

### long is the least correlated variable (0.020092) followed by yr_built (0.052453), and sqft_lot (0.090125)

# Question 2

In [12]:
# I decided to try and add a park shapefile into the mix and treat it similarily to the california coast file. 
#I am not sure how well thought out it was. It ended up making my model worse either way!! This was a fun lab,
# and a creative way to tackle the second half. I appreciated it very much albeit not being able to spend a whole lot
# of time optimizing my model

park = gpd.read_file('/Users/zachary/Desktop/WINTER22/Geog490/geospatial-data-science/labs/lab5/Seattle_parks.shp')
park_utm = park.to_crs('EPSG:32610')

# Compute distance to seattle parks
distance_to_park = []
for i in range(gdf_utm.shape[0]):
    distance_to_park.append(park_utm.distance(gdf_utm['geometry'].iloc[i]).min())
    
# Add to DataFrame
gdf_utm['distance_to_park'] = distance_to_park

In [13]:
# Define feature list
feature_list =  ['sqft_living', 'bathrooms', 'bedrooms', 
                 'lat', 'sqft_lot', 'yr_built', 'long', 'distance_to_park']

# Define features and labels 
X = gdf_utm[feature_list]
y = gdf_utm['price']

# Standarize data
scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)

In [14]:
# Split data 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [15]:
forest_reg = RandomForestRegressor(n_estimators = 30)

# Fit model
forest_reg.fit(X_train, y_train)

RandomForestRegressor(n_estimators=30)

In [16]:
# Predict test labels predictions
predictions = forest_reg.predict(X_test)

# Compute mean-squared-error
final_mse = mean_squared_error(y_test , predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

156876.3619223075